# CS 830 Final Course Project - Sushma Anand Akoju
## KB1

Knowledge base is as follows:
1. Bat is a mammal.
2. Bat is used in baseball.
3. Bats have a barrel.
4. Bat flies with wings.
5. Wings are not barrels.
6. Barrels are not wings.
7. Bats can fly.
8. There is a brown bat.
9. Query: The brown bat can fly.

#### Skolemized Clausal Normal forms generated by Claude 3.7 Sonnet:

1. ¬Bat(x) ∨ Mammal(x)
2. ¬Bat(x) ∨ UsedInBaseball(x)
3. ¬Bat(x) ∨ HasBarrel(x)
4. ¬Bat(x) ∨ HasWings(x)
5. ¬Bat(x) ∨ Flies(x)
6. ¬Wing(x) ∨ ¬Barrel(x)
7. ¬Barrel(x) ∨ ¬Wing(x)
8. ¬Bat(x) ∨ CanFly(x)
9. BrownBat(brown_bat)
10. Bat(brown_bat)
11. ¬CanFly(brown_bat)

CNF grammar, same as given in assignment:

clause → literal | clause
| literal
literal → predicate
| - predicate
predicate → capitalized-name ( term-list )
term-list → term , term-list
| term
term → capitalized-constant-name
| capitalized-function-name ( term-list )
| lowercase-variable-name


LLM did not follow the CNF grammar. The skolemized clausal normal forms need to be modified. So I had to implement an RR algorithm for two set of CNFs generated by the LLM.

In [83]:
from sympy import symbols, Not, Or, And, to_cnf, simplify_logic, parse_expr, Predicate, ask, Symbol
from sympy.parsing.mathematica import parse_mathematica
from sympy.unify.core import unify, unify_var

## Regex Parser to extract clauses and predicates, variables and constants.

In [2]:
raw_clauses = ["¬Bat(x) ∨ Mammal(x)", "¬Bat(x) ∨ UsedInBaseball(x)",
"¬Bat(x) ∨ HasBarrel(x)",
"¬Bat(x) ∨ HasWings(x)",
"¬Bat(x) ∨ Flies(x)",
"¬Wing(x) ∨ ¬Barrel(x)",
"¬Barrel(x) ∨ ¬Wing(x)",
"¬Bat(x) ∨ CanFly(x)",
"BrownBat(Brown_bat)",
"Bat(Brown_bat)",
"¬CanFly(Brown_bat)"]

## Parsing using regex and without Sympy

### Extract predicates using regex

In [3]:
import re
def get_predicate(s):
    if s.startswith('~'):
        return re.findall(r'^\~[A-Za-z0-9]+\(', s)[0][0:-1]
    else:
        return re.findall(r'^[A-Za-z0-9]+\(', s)[0][0:-1]

### Extract arguments from the predicates

In [4]:
def get_pred_arguments(s):
    #return skolem constants/functions and variables
    return re.search(r"\((.*)\)", s).group(1)

In [5]:
get_pred_arguments("BrownBat(Brown_bat)"), get_predicate("BrownBat(Brown_bat)")

('Brown_bat', 'BrownBat')

### Extract skolem constants ad variables from the clauses

In [6]:
def is_skolem_constant(s):
    flag = s and isinstance(s, str) and (s.isupper() or  s[0].isupper())
    return flag

In [7]:
def isvariable(s):
    return isinstance(s, str) and s.islower() and (s.isalnum() or s.isalpha())

In [8]:
isvariable("x1"), is_skolem_constant("Brown_bat")

(True, True)

### Extract clauses that can be recognized by Sympy to supply as input to parsexpr/parsemathematica

In [9]:
clauses_ = []
predicates = []
variables = set()
skolem_constants = set()
skolem_functions = set()
for cl in raw_clauses:
    clause_ = cl.replace("¬", "~").replace(" ", "")
    this_clause = []
    # print(clause_)
    if "∨" in clause_:
        literals = [lit.strip() for lit in clause_.split("∨")]
        # print(literals)
        if len(literals) > 1:
            for lit in literals:
                # print(lit)
                pred = get_predicate(lit)
                predicates.append(pred)
                rest = get_pred_arguments(lit)
                if "(" in rest:
                    skolem_functions.add(rest)
                elif is_skolem_constant(rest):
                    skolem_constants.add(rest)
                else:
                    variables.add(rest)
                # print(pred, rest)
                this_clause.append([pred, rest])
    else:
        pred = get_predicate(clause_)
        predicates.append(pred)
        rest = get_pred_arguments(clause_)
        # print(pred, rest)
        if "(" in rest:
            skolem_functions.add(rest)
        else:
            if is_skolem_constant(rest):
                skolem_constants.add(rest)
        this_clause.append([pred, rest])
    clauses_.append(this_clause)


In [84]:
# clauses_,variables, skolem_constants, skolem_functions

In [10]:
clauses_,

([[['~Bat', 'x'], ['Mammal', 'x']],
  [['~Bat', 'x'], ['UsedInBaseball', 'x']],
  [['~Bat', 'x'], ['HasBarrel', 'x']],
  [['~Bat', 'x'], ['HasWings', 'x']],
  [['~Bat', 'x'], ['Flies', 'x']],
  [['~Wing', 'x'], ['~Barrel', 'x']],
  [['~Barrel', 'x'], ['~Wing', 'x']],
  [['~Bat', 'x'], ['CanFly', 'x']],
  [['BrownBat', 'Brown_bat']],
  [['Bat', 'Brown_bat']],
  [['~CanFly', 'Brown_bat']]],)

In [11]:
# parse_expr('~Bat(x) | ~Brown(x) | ~CanFly(x)')

## My Solution using Sympy's Or which never terminates

In [87]:
Bat1 = Predicate("Bat")
Mammal1 = Predicate("Mammal")
UsedInBaseball1 = Predicate("UsedInBaseball")
HasBarrel1 = Predicate("HasBarrel")
HasWings1 = Predicate("HasWings")
Flies1 = Predicate("Flies")
Wing1 = Predicate("Wing")
Barrel1 = Predicate("Barrel")
CanFly1 = Predicate("CanFly")
BrownBat1 = Predicate("BrownBat")

x = Symbol('x')
Brown_bat = Symbol('brown_bat')

Bat = Bat1(x)
Mammal = Mammal1(x)
UsedInBaseball = UsedInBaseball1(x)
HasBarrel = HasBarrel1(x)
HasWings = HasWings1(x)
Flies = Flies1(x)
Wing = Wing1(x)
Barrel = Barrel1(x)
CanFly = CanFly1(Brown_bat)
BrownBat = BrownBat1(Brown_bat)
Bat2 = Bat1(Brown_bat)

variables = [x]
skolem_constants = [Brown_bat]

In [88]:
Bat.has(Bat1), Bat2.has(Bat1), Bat2.has(Mammal1), Mammal.has(Mammal1), Barrel.has(Barrel1)

(True, True, False, True, True)

In [89]:
Bat1 = Predicate("Bat")
Mammal1 = Predicate("Mammal")
cl = Or(Not(Bat1(x)), Mammal1(x))
to_cnf(cl).args[0].has(x)

True

In [90]:
kb = [
    Or(Not(Bat), Mammal), Or(Not(Bat), UsedInBaseball), Or(Not(Bat), HasBarrel), Or(Not(Bat), HasWings),  Or(Not(Bat), Flies),
    Or(Not(Wing), Not(Barrel)), Or(Not(Barrel), Not(Wing)), Or(Not(Bat), CanFly), BrownBat, Bat2,
]
predicates = [Not(Bat), Mammal, UsedInBaseball, HasBarrel, HasWings, Flies, Not(Barrel), Not(Wing), CanFly, BrownBat, Bat2]
literals = [Bat1,Mammal1,UsedInBaseball1, HasBarrel1, HasWings1, Flies1, Wing1, Barrel1, CanFly1,BrownBat1]
neg_query = Not(CanFly)
kb.append(neg_query)

cnf_clauses = [to_cnf(clause) for clause in kb]

In [91]:
Or(Not(Bat), Mammal).args[1].has(x), Bat2.has_free(skolem_constants[0]), cnf_clauses[5].has(Wing), cnf_clauses[5].has(Not(Wing)), cnf_clauses[0].has(Not(Mammal))

(True, True, True, True, False)

In [92]:
lit_map = {pred: [] for pred in predicates}
for pred in predicates:
    for lit in literals:
        if pred.has(lit):
            lit_map[pred].append(lit)

In [93]:
lit_map

{~Q.Bat(x): [Q.Bat],
 Q.Mammal(x): [Q.Mammal],
 Q.UsedInBaseball(x): [Q.UsedInBaseball],
 Q.HasBarrel(x): [Q.HasBarrel],
 Q.HasWings(x): [Q.HasWings],
 Q.Flies(x): [Q.Flies],
 ~Q.Barrel(x): [Q.Barrel],
 ~Q.Wing(x): [Q.Wing],
 Q.CanFly(brown_bat): [Q.CanFly],
 Q.BrownBat(brown_bat): [Q.BrownBat],
 Q.Bat(brown_bat): [Q.Bat]}

In [94]:
cnf_clauses

[Q.Mammal(x) | ~Q.Bat(x),
 Q.UsedInBaseball(x) | ~Q.Bat(x),
 Q.HasBarrel(x) | ~Q.Bat(x),
 Q.HasWings(x) | ~Q.Bat(x),
 Q.Flies(x) | ~Q.Bat(x),
 ~Q.Barrel(x) | ~Q.Wing(x),
 ~Q.Barrel(x) | ~Q.Wing(x),
 Q.CanFly(brown_bat) | ~Q.Bat(x),
 Q.BrownBat(brown_bat),
 Q.Bat(brown_bat),
 ~Q.CanFly(brown_bat)]

In [95]:
cnf_map = {predicate: [] for predicate in predicates}
pred_map = {clause: [] for clause in cnf_clauses}
for predicate in predicates:
    for cnf in cnf_clauses:
        # print(predicate, cnf)
        if cnf.has(predicate):
            cnf_map[predicate].append(cnf)
            pred_map[cnf].append(predicate)
        # else:
            # print(predicate, cnf)

In [96]:
pred_map

{Q.Mammal(x) | ~Q.Bat(x): [~Q.Bat(x), Q.Mammal(x)],
 Q.UsedInBaseball(x) | ~Q.Bat(x): [~Q.Bat(x), Q.UsedInBaseball(x)],
 Q.HasBarrel(x) | ~Q.Bat(x): [~Q.Bat(x), Q.HasBarrel(x)],
 Q.HasWings(x) | ~Q.Bat(x): [~Q.Bat(x), Q.HasWings(x)],
 Q.Flies(x) | ~Q.Bat(x): [~Q.Bat(x), Q.Flies(x)],
 ~Q.Barrel(x) | ~Q.Wing(x): [~Q.Barrel(x),
  ~Q.Barrel(x),
  ~Q.Wing(x),
  ~Q.Wing(x)],
 Q.CanFly(brown_bat) | ~Q.Bat(x): [~Q.Bat(x), Q.CanFly(brown_bat)],
 Q.BrownBat(brown_bat): [Q.BrownBat(brown_bat)],
 Q.Bat(brown_bat): [Q.Bat(brown_bat)],
 ~Q.CanFly(brown_bat): [Q.CanFly(brown_bat)]}

In [97]:
for k,v in cnf_map.items():
    if Not(k) in v:
        print(k, v)

Q.CanFly(brown_bat) [Q.CanFly(brown_bat) | ~Q.Bat(x), ~Q.CanFly(brown_bat)]


In [98]:
cnf_lit_map = {literal: [] for literal in literals}
for literal in literals:
    for cnf in cnf_clauses:
        # print(predicate, cnf)
        if cnf.has(literal):
            cnf_lit_map[literal].append(cnf)
        # else:
            # print(predicate, cnf)


In [99]:
cnf_lit_map

{Q.Bat: [Q.Mammal(x) | ~Q.Bat(x),
  Q.UsedInBaseball(x) | ~Q.Bat(x),
  Q.HasBarrel(x) | ~Q.Bat(x),
  Q.HasWings(x) | ~Q.Bat(x),
  Q.Flies(x) | ~Q.Bat(x),
  Q.CanFly(brown_bat) | ~Q.Bat(x),
  Q.Bat(brown_bat)],
 Q.Mammal: [Q.Mammal(x) | ~Q.Bat(x)],
 Q.UsedInBaseball: [Q.UsedInBaseball(x) | ~Q.Bat(x)],
 Q.HasBarrel: [Q.HasBarrel(x) | ~Q.Bat(x)],
 Q.HasWings: [Q.HasWings(x) | ~Q.Bat(x)],
 Q.Flies: [Q.Flies(x) | ~Q.Bat(x)],
 Q.Wing: [~Q.Barrel(x) | ~Q.Wing(x), ~Q.Barrel(x) | ~Q.Wing(x)],
 Q.Barrel: [~Q.Barrel(x) | ~Q.Wing(x), ~Q.Barrel(x) | ~Q.Wing(x)],
 Q.CanFly: [Q.CanFly(brown_bat) | ~Q.Bat(x), ~Q.CanFly(brown_bat)],
 Q.BrownBat: [Q.BrownBat(brown_bat)]}

In [100]:
cl = cnf_clauses[10]
cl.atoms(), cl.args, cnf_clauses[10].is_Not, cnf_clauses[9].is_Not, isinstance(cnf_clauses[0], Or), isinstance(cnf_clauses[10], Or), cnf_clauses[0].args[0].has(x)

({CanFly, brown_bat}, (Q.CanFly(brown_bat),), True, False, True, False, True)

In [101]:
cnf_clauses[10].has(Not(CanFly)), cnf_clauses[10].has(cnf_clauses[10]), cnf_clauses[10].has(cnf_clauses[10].args[0]), cnf_clauses[0].has(cnf_clauses[0])

(True, True, True, True)

In [102]:
cnf_clauses[10].equals(cnf_clauses[1]), cnf_clauses[10].is_Not, cnf_clauses[10].args, cnf_clauses[0].args[1].args[0].args[0].equals(Bat1)

(False, True, (Q.CanFly(brown_bat),), True)

In [103]:
#helper fucntions

def has_variable(literal, variable):
    return literal.has(variable)

def has_skolem_constant(literal, skolem_constant):
    return literal.has(skolem_constant)

def get_clause_args(clause):
    return clause.args

# def get_matching_litvar(clause, cnf_clauses, variables):
#     matching_literals = {}
#     matching_literals[clause] = {}

#     for pred in clause.args:
#         if pred in cnf_map:
#             clauses = cnf_map[pred]
#             for cnf in clauses:
#                 if not cnf.equals(clause):
#                     if cnf.has(pred):


#                     if pred not in matching_literals[clause]:
#                         matching_literals[clause][pred] = []
#                     matching_literals[clause][pred].append(cnf)
#                 elif cnf.has(Not(pred)):
#                     if Not(pred) not in matching_literals[clause]:
#                         matching_literals[clause][Not(pred)] = []
#                     matching_literals[clause][Not(pred)].append(cnf)

In [104]:
def can_resolve(clause1, clause2, pred_map):
    for lit1 in pred_map[clause1]:
        if isinstance(lit1, Not) or lit1.is_Not:
            pred = lit1.args[0].args[0] #first returns Not(this_lit), access this_lit
            if clause2.has(pred):
                return True
        else:
            pred = Not(lit1)
            if clause2.has(pred):
                return True
    return False

def update_pred_map(new_clause_literals, new_clause, pred_map):
    for lit in new_clause_literals:
        if new_clause not in pred_map:
            pred_map[new_clause] = []
        pred_map[new_clause].append(lit)
    return pred_map


In [105]:
def resolve(clause1, clause2, pred_map):
    resolvents = set()
    for lit1 in pred_map[clause1]:
        if isinstance(lit1, Not) or lit1.is_Not:
            pred = lit1.args[0].args[0] #first returns Not(this_lit), access this_lit
            if clause2.has(pred):
                literals1 = pred_map[clause1]
                literals2 = pred_map[clause2]
                new_clause = tuple(l for l in literals1 if not l.equals(lit1)) + tuple(l for l in literals2 if not l.equals(pred) )
                # print("newclause",len(new_clause), new_clause)
                or_new_clause = Or(*list(new_clause)) #tuple(set(new_clause))
                update_pred_map(list(new_clause), or_new_clause, pred_map)
                resolvents.add(to_cnf(or_new_clause))
        else:
            not_pred = Not(lit1)
            if clause2.has(not_pred):
                literals1 = pred_map[clause1]
                literals2 = pred_map[clause2]
                new_clause = tuple(l for l in literals1 if not l.equals(lit1)) + tuple(l for l in literals2 if not l.equals(not_pred) )
                # print("newclause",len(new_clause), new_clause)
                or_new_clause = Or(*list(new_clause)) #tuple(set(new_clause))
                update_pred_map(list(new_clause), or_new_clause, pred_map)
                # new_clause = tuple(set(new_clause))
                resolvents.add(to_cnf(or_new_clause))
    return resolvents

In [106]:
def resolution_refutation(clauses, pred_map):
    allclauses = set(clauses)
    newclauses = set(clauses)
    while True:
        generated = set()
        twoclauses_list = [(c1, c2) for c1 in allclauses for c2 in newclauses if c1 != c2 and can_resolve(c1,c2, pred_map)]
        # print(twoclauses_list)
        newclauses = set()
        for c1, c2 in twoclauses_list:
            resolvents = resolve(c1, c2, pred_map)
            # print("*********************")
            # print(resolvents)
            # print(clauses)
            for res in resolvents:
                if res == ():
                    #empty clause
                    return True
                if res not in allclauses:
                    newclauses.add(res)
                    generated.add(res)
        if generated.issubset(allclauses):
            return False
        allclauses.update(newclauses)


In [ ]:
# resolution_refutation(cnf_clauses, pred_map)

In [ ]:
# def check_subs(lit1, lit2, variables, skolem_constants, skolem_functions):
#     for var in variables:
#         if lit1.has(var) and not lit2.has(var):
#             pred = lit1.args[0]



In [171]:
# def resolve1(clause1, clause2, pred_map, variable, skolem_constant):
#     resolvents = set()
#     for lit1 in pred_map[clause1]:
#         if isinstance(lit1, Not) or lit1.is_Not:
#             print(lit1.args)
#             pred = lit1.args[0].args[0] #first returns Not(this_lit), access this_lit
#             if pred in clause2:
#                 literals1 = pred_map[clause1]
#                 literals2 = pred_map[clause2]
#                 new_clause = tuple(l for l in literals1 if not l.equals(lit1)) + tuple(l for l in literals2 if not l.equals(pred) )
#                 # print("newclause",len(new_clause), new_clause)
#                 update_pred_map(new_clause, pred_map)
#                 new_clause = tuple(set(new_clause))
#                 resolvents.add(new_clause)
#         else:
#             not_pred = Not(lit1)
#             if not_pred in clause2:
#                 literals1 = pred_map[clause1]
#                 literals2 = pred_map[clause2]
#                 new_clause = tuple(l for l in literals1 if not l.equals(lit1)) + tuple(l for l in literals2 if not l.equals(not_pred) )
#                 # print("newclause",len(new_clause), new_clause)
#                 # or_new_clause = Or(*list(new_clause)) #tuple(set(new_clause))
#                 update_pred_map(new_clause, pred_map)
#                 new_clause = tuple(set(new_clause))
#                 resolvents.add(new_clause)
#             elif variable in not_pred and skolem_constant:
#                 literals1 = pred_map[clause1]
#                 literals2 = pred_map[clause2]
#                  for lit in clause2:
#                     if skolem_constants[0] in lit:
#                         new_clause = tuple(l for l in literals1 if not l.equals(lit1)) + tuple(l for l in literals2 if not l.equals(not_pred) )
#                 #check skolem constants
#                 literals2 = pred_map[clause2]
#     return resolvents

## My Solution using Sympy without using Or but instead make clause as a tuple like in the assignment

### Manually fill in the predicates since CNF grammar is different

In [384]:
Bat1 = Predicate("Bat")
Mammal1 = Predicate("Mammal")
UsedInBaseball1 = Predicate("UsedInBaseball")
HasBarrel1 = Predicate("HasBarrel")
HasWings1 = Predicate("HasWings")
Flies1 = Predicate("Flies")
Wing1 = Predicate("Wing")
Barrel1 = Predicate("Barrel")
CanFly1 = Predicate("CanFly")
BrownBat1 = Predicate("BrownBat")

x = Symbol('x')
Brown_bat = Symbol('brown_bat')

Bat = Bat1(x)
Mammal = Mammal1(x)
UsedInBaseball = UsedInBaseball1(x)
HasBarrel = HasBarrel1(x)
HasWings = HasWings1(x)
Flies = Flies1(x)
Wing = Wing1(x)
Barrel = Barrel1(x)
CanFly2 = CanFly1(x)
CanFly = CanFly1(Brown_bat)
BrownBat = BrownBat1(Brown_bat)
Bat2 = Bat1(Brown_bat)

variables = [x]
skolem_constants = [Brown_bat]

### Run some tests to checl predicates containment in Applied predicates

In [13]:
Bat.has(Bat1), Bat2.has(Bat1), Bat2.has(Mammal1), Mammal.has(Mammal1), Barrel.has(Barrel1)

(True, True, False, True, True)

### Convert to CNF to test validity of predicates

In [14]:
Bat1 = Predicate("Bat")
Mammal1 = Predicate("Mammal")
cl = Or(Not(Bat1(x)), Mammal1(x))
to_cnf(cl).args[0].has(x)

True

### Generate KB with clauses using Applied *predicates*

In [385]:
kb = [
    (Not(Bat), Mammal), (Not(Bat), UsedInBaseball), (Not(Bat), HasBarrel), (Not(Bat), HasWings),  (Not(Bat), Flies),
    (Not(Wing), Not(Barrel)), (Not(Barrel), Not(Wing)), (Not(Bat), CanFly2), (BrownBat,), (Bat2,),
]
predicates = [Not(Bat), Mammal, UsedInBaseball, HasBarrel, HasWings, Flies, Not(Barrel), Not(Wing), CanFly2, BrownBat, Bat2, Not(CanFly)]
literals = [Bat1,Mammal1,UsedInBaseball1, HasBarrel1, HasWings1, Flies1, Wing1, Barrel1, CanFly1,BrownBat1]
neg_query = Not(CanFly)
kb.append((neg_query,))

# cnf_clauses = [to_cnf(clause) for clause in kb]

In [386]:
predicates

[~Q.Bat(x),
 Q.Mammal(x),
 Q.UsedInBaseball(x),
 Q.HasBarrel(x),
 Q.HasWings(x),
 Q.Flies(x),
 ~Q.Barrel(x),
 ~Q.Wing(x),
 Q.CanFly(x),
 Q.BrownBat(brown_bat),
 Q.Bat(brown_bat),
 ~Q.CanFly(brown_bat)]

#### Map each predicate with corresponding literal

In [387]:
lit_map = {pred: [] for pred in predicates}
for pred in predicates:
    for lit in literals:
        if pred.has(lit):
            lit_map[pred].append(lit)

In [388]:
lit_map

{~Q.Bat(x): [Q.Bat],
 Q.Mammal(x): [Q.Mammal],
 Q.UsedInBaseball(x): [Q.UsedInBaseball],
 Q.HasBarrel(x): [Q.HasBarrel],
 Q.HasWings(x): [Q.HasWings],
 Q.Flies(x): [Q.Flies],
 ~Q.Barrel(x): [Q.Barrel],
 ~Q.Wing(x): [Q.Wing],
 Q.CanFly(x): [Q.CanFly],
 Q.BrownBat(brown_bat): [Q.BrownBat],
 Q.Bat(brown_bat): [Q.Bat],
 ~Q.CanFly(brown_bat): [Q.CanFly]}

### kb_clauses

In [389]:
kb

[(~Q.Bat(x), Q.Mammal(x)),
 (~Q.Bat(x), Q.UsedInBaseball(x)),
 (~Q.Bat(x), Q.HasBarrel(x)),
 (~Q.Bat(x), Q.HasWings(x)),
 (~Q.Bat(x), Q.Flies(x)),
 (~Q.Wing(x), ~Q.Barrel(x)),
 (~Q.Barrel(x), ~Q.Wing(x)),
 (~Q.Bat(x), Q.CanFly(x)),
 (Q.BrownBat(brown_bat),),
 (Q.Bat(brown_bat),),
 (~Q.CanFly(brown_bat),)]

### Map predicates to clauses

In [390]:
kb_map = {predicate: [] for predicate in predicates}
pred_map = {clause: [] for clause in kb}
for predicate in predicates:
    for cnf in kb:
        # print(predicate, cnf)
        if predicate in cnf:
            # print("True")
            kb_map[predicate].append(cnf)
            pred_map[cnf].append(predicate)
        # else:
            # print(predicate, cnf)

In [393]:
pred_map

{(~Q.Bat(x), Q.Mammal(x)): [~Q.Bat(x), Q.Mammal(x)],
 (~Q.Bat(x), Q.UsedInBaseball(x)): [~Q.Bat(x), Q.UsedInBaseball(x)],
 (~Q.Bat(x), Q.HasBarrel(x)): [~Q.Bat(x), Q.HasBarrel(x)],
 (~Q.Bat(x), Q.HasWings(x)): [~Q.Bat(x), Q.HasWings(x)],
 (~Q.Bat(x), Q.Flies(x)): [~Q.Bat(x), Q.Flies(x)],
 (~Q.Wing(x), ~Q.Barrel(x)): [~Q.Barrel(x), ~Q.Wing(x)],
 (~Q.Barrel(x), ~Q.Wing(x)): [~Q.Barrel(x), ~Q.Wing(x)],
 (~Q.Bat(x), Q.CanFly(x)): [~Q.Bat(x), Q.CanFly(x)],
 (Q.BrownBat(brown_bat),): [Q.BrownBat(brown_bat)],
 (Q.Bat(brown_bat),): [Q.Bat(brown_bat)],
 (~Q.CanFly(brown_bat),): [~Q.CanFly(brown_bat)]}

### Map clauses to predicate, argument pairs of each clause as a list
### Map predicates with arguments

In [391]:
kb_map = {predicate: {} for predicate in predicates}
pred_args = {predicate: {} for predicate in predicates}
for predicate in predicates:
    for cnf in kb:
        # print(predicate, cnf)
        if predicate in cnf:
            # print("True")
            this_cnf = []
            for lit in cnf:
                # if lit not in kb_map[predicate]:
                #     kb_map[predicate][lit] = []
                if isinstance(lit, Not) or lit.is_Not:
                    pred = lit.args[0].args[0]
                    arg = lit.args[0].args[1]
                    this_cnf.append([pred, arg])
                    if lit not in pred_args[predicate]:
                        pred_args[predicate][lit] = []
                    if [pred, arg] not in pred_args[predicate][lit]:
                        pred_args[predicate][lit].append([pred, arg])

                else:
                    pred = lit.args[0]
                    arg = lit.args[1]
                    this_cnf.append([pred, arg])
                    if lit not in pred_args[predicate]:
                        pred_args[predicate][lit] = []
                    if [pred, arg] not in pred_args[predicate][lit]:
                        pred_args[predicate][lit].append([pred, arg])
                # print(this_cnf, lit, kb_map[predicate],lit in kb_map[predicate])
                kb_map[predicate][cnf] = this_cnf



In [392]:
kb_map

{~Q.Bat(x): {(~Q.Bat(x), Q.Mammal(x)): [[Q.Bat, x], [Q.Mammal, x]],
  (~Q.Bat(x), Q.UsedInBaseball(x)): [[Q.Bat, x], [Q.UsedInBaseball, x]],
  (~Q.Bat(x), Q.HasBarrel(x)): [[Q.Bat, x], [Q.HasBarrel, x]],
  (~Q.Bat(x), Q.HasWings(x)): [[Q.Bat, x], [Q.HasWings, x]],
  (~Q.Bat(x), Q.Flies(x)): [[Q.Bat, x], [Q.Flies, x]],
  (~Q.Bat(x), Q.CanFly(x)): [[Q.Bat, x], [Q.CanFly, x]]},
 Q.Mammal(x): {(~Q.Bat(x), Q.Mammal(x)): [[Q.Bat, x], [Q.Mammal, x]]},
 Q.UsedInBaseball(x): {(~Q.Bat(x), Q.UsedInBaseball(x)): [[Q.Bat, x],
   [Q.UsedInBaseball, x]]},
 Q.HasBarrel(x): {(~Q.Bat(x), Q.HasBarrel(x)): [[Q.Bat, x], [Q.HasBarrel, x]]},
 Q.HasWings(x): {(~Q.Bat(x), Q.HasWings(x)): [[Q.Bat, x], [Q.HasWings, x]]},
 Q.Flies(x): {(~Q.Bat(x), Q.Flies(x)): [[Q.Bat, x], [Q.Flies, x]]},
 ~Q.Barrel(x): {(~Q.Wing(x), ~Q.Barrel(x)): [[Q.Wing, x], [Q.Barrel, x]],
  (~Q.Barrel(x), ~Q.Wing(x)): [[Q.Barrel, x], [Q.Wing, x]]},
 ~Q.Wing(x): {(~Q.Wing(x), ~Q.Barrel(x)): [[Q.Wing, x], [Q.Barrel, x]],
  (~Q.Barrel(x), ~Q.W

In [394]:
pred_args

{~Q.Bat(x): {~Q.Bat(x): [[Q.Bat, x]],
  Q.Mammal(x): [[Q.Mammal, x]],
  Q.UsedInBaseball(x): [[Q.UsedInBaseball, x]],
  Q.HasBarrel(x): [[Q.HasBarrel, x]],
  Q.HasWings(x): [[Q.HasWings, x]],
  Q.Flies(x): [[Q.Flies, x]],
  Q.CanFly(x): [[Q.CanFly, x]]},
 Q.Mammal(x): {~Q.Bat(x): [[Q.Bat, x]], Q.Mammal(x): [[Q.Mammal, x]]},
 Q.UsedInBaseball(x): {~Q.Bat(x): [[Q.Bat, x]],
  Q.UsedInBaseball(x): [[Q.UsedInBaseball, x]]},
 Q.HasBarrel(x): {~Q.Bat(x): [[Q.Bat, x]], Q.HasBarrel(x): [[Q.HasBarrel, x]]},
 Q.HasWings(x): {~Q.Bat(x): [[Q.Bat, x]], Q.HasWings(x): [[Q.HasWings, x]]},
 Q.Flies(x): {~Q.Bat(x): [[Q.Bat, x]], Q.Flies(x): [[Q.Flies, x]]},
 ~Q.Barrel(x): {~Q.Wing(x): [[Q.Wing, x]], ~Q.Barrel(x): [[Q.Barrel, x]]},
 ~Q.Wing(x): {~Q.Wing(x): [[Q.Wing, x]], ~Q.Barrel(x): [[Q.Barrel, x]]},
 Q.CanFly(x): {~Q.Bat(x): [[Q.Bat, x]], Q.CanFly(x): [[Q.CanFly, x]]},
 Q.BrownBat(brown_bat): {Q.BrownBat(brown_bat): [[Q.BrownBat, brown_bat]]},
 Q.Bat(brown_bat): {Q.Bat(brown_bat): [[Q.Bat, brown_bat]

In [395]:
Not(Bat1(x)).args[0].args

(Q.Bat, x)

### Notice that Not(Not predicate) is not same as predicate without negation as it Q.predicate but Applied predicates are different.



In [396]:
for k,v in kb_map.items():
    if Not(k) in v:
        print(k, v)

### CNF literal to CNF map

In [397]:
cnf_lit_map = {literal: [] for literal in literals}
for literal in literals:
    for cnf in kb:
        # print(literal, str(cnf[0]))
        for lit in cnf:
            if str(literal) in str(lit):
                cnf_lit_map[literal].append(cnf)
        # else:
            # print(predicate, cnf)


In [398]:
cnf_lit_map

{Q.Bat: [(~Q.Bat(x), Q.Mammal(x)),
  (~Q.Bat(x), Q.UsedInBaseball(x)),
  (~Q.Bat(x), Q.HasBarrel(x)),
  (~Q.Bat(x), Q.HasWings(x)),
  (~Q.Bat(x), Q.Flies(x)),
  (~Q.Bat(x), Q.CanFly(x)),
  (Q.Bat(brown_bat),)],
 Q.Mammal: [(~Q.Bat(x), Q.Mammal(x))],
 Q.UsedInBaseball: [(~Q.Bat(x), Q.UsedInBaseball(x))],
 Q.HasBarrel: [(~Q.Bat(x), Q.HasBarrel(x))],
 Q.HasWings: [(~Q.Bat(x), Q.HasWings(x))],
 Q.Flies: [(~Q.Bat(x), Q.Flies(x))],
 Q.Wing: [(~Q.Wing(x), ~Q.Barrel(x)), (~Q.Barrel(x), ~Q.Wing(x))],
 Q.Barrel: [(~Q.Wing(x), ~Q.Barrel(x)), (~Q.Barrel(x), ~Q.Wing(x))],
 Q.CanFly: [(~Q.Bat(x), Q.CanFly(x)), (~Q.CanFly(brown_bat),)],
 Q.BrownBat: [(Q.BrownBat(brown_bat),)]}

### Since constants, variables were already extracted during parsing, check if a given input has a variable, has a constant or is a predicate or variable or constant.

In [399]:
#helper fucntions.

def has_variable(literal, variable):
    return literal.has(variable)

def has_skolem_constant(literal, skolem_constant):
    return literal.has(skolem_constant)

def is_variable(variable, variables):
    return True if variable in variables else False

def is_skolem_constant(constant, skolem_constants):
    return True if constant in skolem_constants else False

def is_predicate(predicate, predicates):
    return True if predicate in predicates else False



### Looking for complementary predicates of Sympy
#### Sympy's Applied predicates have arguments that do not retain the Not, Or conditionals.
#### Check each part of clause for itself: predicate, argument only after checking if topmost node in the tree is a Not operator and vice versa.

In [400]:
def is_complement(lit1, lit2):
    if (isinstance(lit1, Not) or lit1.is_Not) and not (isinstance(lit2, Not) or lit2.is_Not):
        pred1 = lit1.args[0].args[0] #first returns Not(this_lit), access this_lit
        arg1 = lit1.args[0].args[1]
        pred2 = lit2.args[0] #first returns Not(this_lit), access this_lit
        arg2 = lit2.args[1]
        if pred2.has(pred1): #and arg1.has(arg2):
            return True
    elif ( isinstance(lit2, Not) or lit2.is_Not) and not (isinstance(lit1, Not) or lit1.is_Not):
        pred1 = lit1.args[0] #first returns Not(this_lit), access this_lit
        arg1 = lit1.args[1]
        pred2 = lit2.args[0].args[0] #first returns Not(this_lit), access this_lit
        arg2 = lit2.args[0].args[1]
        if pred2.has(pred1): #and arg1.has(arg2):
            return True
    return False

def is_negation(clause1, clause2, pred_map):
    for lit1 in pred_map[clause1]:
        for lit2 in pred_map[clause2]:
            if is_complement(lit1, lit2):
                return True
    return False

In [77]:
# Bat1(Brown_bat).has(Not(Not(Bat1(x)).args[0].args[0])), Bat1(Brown_bat).has(Not(Bat1(x)).args[0].args[0]), Bat1(Brown_bat).has(Not(Bat1(x)).args[0].args[0]),Not(Not(Bat1(x)).args[0].args[0])

In [401]:
is_negation(kb[0], kb[9], pred_map),kb[0][0], kb[9][0] #different arguments, complementary pair of predicates

(True, ~Q.Bat(x), Q.Bat(brown_bat))

In [402]:
kb[10][0].args[0].args

(Q.CanFly, brown_bat)

In [403]:
is_complement(Bat1(Brown_bat), Not(Bat1(x))), is_complement(Not(Bat1(Brown_bat)), Not(Bat1(x))), is_complement(Not(Bat1(Brown_bat)), Bat1(x)), is_complement(Bat1(Brown_bat), Bat1(x))

(True, False, True, False)

In [416]:
clauses = clauses_map.keys()
allclauses = set(clauses)
newclauses = set(clauses)

for c1 in allclauses:
    for c2 in newclauses:
        if c1 != c2:
            print(c1,c2,is_negation(c1, c2, pred_map))

(~Q.Bat(x), Q.Mammal(x)) (~Q.Bat(x), Q.HasWings(x)) False
(~Q.Bat(x), Q.Mammal(x)) (~Q.Bat(x), Q.CanFly(x)) False
(~Q.Bat(x), Q.Mammal(x)) (~Q.Wing(x), ~Q.Barrel(x)) False
(~Q.Bat(x), Q.Mammal(x)) (~Q.Bat(x), Q.Flies(x)) False
(~Q.Bat(x), Q.Mammal(x)) (Q.Bat(brown_bat),) True
(~Q.Bat(x), Q.Mammal(x)) (~Q.Bat(x), Q.HasBarrel(x)) False
(~Q.Bat(x), Q.Mammal(x)) (~Q.Barrel(x), ~Q.Wing(x)) False
(~Q.Bat(x), Q.Mammal(x)) (~Q.CanFly(brown_bat),) False
(~Q.Bat(x), Q.Mammal(x)) (Q.BrownBat(brown_bat),) False
(~Q.Bat(x), Q.Mammal(x)) (~Q.Bat(x), Q.UsedInBaseball(x)) False
(~Q.Bat(x), Q.HasWings(x)) (~Q.Bat(x), Q.Mammal(x)) False
(~Q.Bat(x), Q.HasWings(x)) (~Q.Bat(x), Q.CanFly(x)) False
(~Q.Bat(x), Q.HasWings(x)) (~Q.Wing(x), ~Q.Barrel(x)) False
(~Q.Bat(x), Q.HasWings(x)) (~Q.Bat(x), Q.Flies(x)) False
(~Q.Bat(x), Q.HasWings(x)) (Q.Bat(brown_bat),) True
(~Q.Bat(x), Q.HasWings(x)) (~Q.Bat(x), Q.HasBarrel(x)) False
(~Q.Bat(x), Q.HasWings(x)) (~Q.Barrel(x), ~Q.Wing(x)) False
(~Q.Bat(x), Q.HasWings(x)

### Unify and Substitute implementation

In [445]:
def update_pred_map(new_clause, pred_map):
    if new_clause not in pred_map:
        pred_map[new_clause] = []
    for lit in new_clause:
        pred_map[new_clause].append(lit)
    return pred_map

def update_clauses_map(new_clause, clauses_map):

    this_cnf = []
    for lit in new_clause:
        # if lit not in kb_map[predicate]:
        #     kb_map[predicate][lit] = []
        if isinstance(lit, Not) or lit.is_Not:
            pred = lit.args[0].args[0]
            arg = lit.args[0].args[1]
            this_cnf.append([pred, arg])
        else:
            pred = lit.args[0]
            arg = lit.args[1]
            this_cnf.append([pred, arg])
            if lit not in pred_args[predicate]:
                pred_args[predicate][lit] = []
            if [pred, arg] not in pred_args[predicate][lit]:
                pred_args[predicate][lit].append([pred, arg])
        # print(this_cnf, lit, kb_map[predicate],lit in kb_map[predicate])
    clauses_map[new_clause] = this_cnf
    return clauses_map



def unify(x, y, s={}):
    # print(x,y)
    if isinstance(s, bool) and s is False:
        # print("False")
        return False
    if x == y:
        return s
    if x in variables or x in skolem_constants:
        # print("x",x)
        return unify_variable(x, y, s)
    if is_variable(y, variables) or is_skolem_constant(y, skolem_constants):
        # print("y",y)
        return unify_variable(y, x, s)
    if not isinstance(x, list) or not isinstance(y, list):
        # print(x,y)
        return False
    if len(x) != len(y):
        # print("length", x,y)
        return False
    return unify(x[1:], y[1:], unify(x[0],y[0], s))

def unify_variable(var, x, s):
    if var in s:
        return unify(s[var], x, s)
    if x in s:
        return unify(var, s[x], s)
    if occurs_check(var, x,s):
        return False
    s[var] = x
    return s

def occurs_check(var, x, s):
    if var == x:
        return True
    if is_variable(x,variables) and x in s:
        return occurs_check(var, s[x], s)
    if isinstance(x, list):
        any(occurs_check(var, term, s) for term in x)
    return False

def substitute(s, expr, variables):
    if is_variable(expr, variables):
        return s.get(expr, expr)
    if isinstance(expr, list):
        return [substitute(s,exp, variables) for exp in expr]
    return expr


In [405]:
substitute({x:Brown_bat}, [Bat1,x], variables)

[Q.Bat, brown_bat]

In [425]:
clauses_map = {}
for k,clause in kb_map.items():
    for k1,v in clause.items():
        clauses_map[k1] = v

In [407]:
clauses_map

{(~Q.Bat(x), Q.Mammal(x)): [[Q.Bat, x], [Q.Mammal, x]],
 (~Q.Bat(x), Q.UsedInBaseball(x)): [[Q.Bat, x], [Q.UsedInBaseball, x]],
 (~Q.Bat(x), Q.HasBarrel(x)): [[Q.Bat, x], [Q.HasBarrel, x]],
 (~Q.Bat(x), Q.HasWings(x)): [[Q.Bat, x], [Q.HasWings, x]],
 (~Q.Bat(x), Q.Flies(x)): [[Q.Bat, x], [Q.Flies, x]],
 (~Q.Bat(x), Q.CanFly(x)): [[Q.Bat, x], [Q.CanFly, x]],
 (~Q.Wing(x), ~Q.Barrel(x)): [[Q.Wing, x], [Q.Barrel, x]],
 (~Q.Barrel(x), ~Q.Wing(x)): [[Q.Barrel, x], [Q.Wing, x]],
 (Q.BrownBat(brown_bat),): [[Q.BrownBat, brown_bat]],
 (Q.Bat(brown_bat),): [[Q.Bat, brown_bat]],
 (~Q.CanFly(brown_bat),): [[Q.CanFly, brown_bat]]}

In [444]:
for k, clause1 in clauses_map.items():
    for j,clause2 in clauses_map.items():
        for lit1 in clause1:
            for lit2 in clause2:
                if lit1 != lit2:
                    res = unify(lit1, lit2)
                    if res is not False:
                        print(lit1, lit2, res)

[Q.Bat, x] [Q.Bat, brown_bat] {x: brown_bat}
[Q.Bat, x] [Q.Bat, brown_bat] {x: brown_bat}
[Q.Bat, x] [Q.Bat, brown_bat] {x: brown_bat}
[Q.Bat, x] [Q.Bat, brown_bat] {x: brown_bat}
[Q.Bat, x] [Q.Bat, brown_bat] {x: brown_bat}
[Q.Bat, x] [Q.Bat, brown_bat] {x: brown_bat}
[Q.CanFly, x] [Q.CanFly, brown_bat] {x: brown_bat}
[Q.Bat, brown_bat] [Q.Bat, x] {x: brown_bat}
[Q.Bat, brown_bat] [Q.Bat, x] {x: brown_bat}
[Q.Bat, brown_bat] [Q.Bat, x] {x: brown_bat}
[Q.Bat, brown_bat] [Q.Bat, x] {x: brown_bat}
[Q.Bat, brown_bat] [Q.Bat, x] {x: brown_bat}
[Q.Bat, brown_bat] [Q.Bat, x] {x: brown_bat}
[Q.Bat, brown_bat] [Q.Bat, x] {x: brown_bat}
[Q.CanFly, brown_bat] [Q.CanFly, x] {x: brown_bat}
[Q.CanFly, brown_bat] [Q.CanFly, x] {x: brown_bat}
[Q.CanFly, x] [Q.CanFly, brown_bat] {x: brown_bat}
[Q.Bat, x] [Q.Bat, brown_bat] {x: brown_bat}


### Resolve and return resolvents and update predicate & clause maps respctively.

In [442]:
def can_resolve(clause1, clause2, pred_map):
    for lit1 in pred_map[clause1]:
        if isinstance(lit1, Not) or lit1.is_Not:
            pred = lit1.args[0].args[0] #first returns Not(this_lit), access this_lit
            if pred in clause2:
                return True
        else:
            pred = Not(lit1)
            if pred in clause2:
                return True
    return False

def resolve(clause1, clause2, pred_map, clauses_map):
    resolvents = set()
    for lit1, lit1_list in zip(clause1, clauses_map[clause1]):
        for lit2, lit2_list in zip(clause2, clauses_map[clause2]):
            # print(lit1, lit2, clause1, clause2)
            if is_complement(lit1, lit2):
                #unify
                # print("resolve")
                sub = unify(lit1_list,lit2_list)
                print("substitution",sub)
                if sub is not None:
                    for l in clause1:
                        if l != lit1:
                            print(substitute(sub,l, variables))
                    clause1_new = [substitute(sub,l, variables) for l in clause1 if l != lit1]
                    clause2_new = [substitute( sub, l,variables) for l in clause2 if l != lit2]
                    # print(clause1_new, clause2_new)
                    new_clause = set(clause1_new + clause2_new)
                    update_pred_map(tuple(new_clause), pred_map)
                    update_clauses_map(tuple(new_clause), clauses_map)
                    resolvents.add(tuple(new_clause))
    return resolvents

### Resolution Refutation

In [446]:
def resolution_refutation(clauses, pred_map, clauses_map):
    clauses = clauses_map.keys()
    allclauses = set(clauses)
    newclauses = set(clauses)
    while True:
        generated = set()
        clauses_list = [(c1, c2) for c1 in allclauses for c2 in newclauses if c1 != c2 and is_negation(c1, c2, pred_map)]
        # print(clauses_list)
        newclauses = set()
        results = []
        resolvents = None
        for c1, c2 in clauses_list:
            resolvents = resolve(c1, c2, pred_map, clauses_map)
            results.extend(resolvents)
            # print("*********************")
            # print(resolvents)
            # print(clauses)
            for res in resolvents:
                if res == ():
                    #empty clause
                    return True
                if res not in allclauses:
                    newclauses.add(res)
                    generated.add(res)
        if generated.issubset(allclauses):
            return False, resolvents, generated
        allclauses.update(newclauses)


In [411]:
temp =list(clauses_map.keys())
print(temp[0], temp[9])
can_resolve(temp[9], temp[0], pred_map)

(~Q.Bat(x), Q.Mammal(x)) (Q.Bat(brown_bat),)


False

In [447]:
resolution_refutation(clauses_map, pred_map, clauses_map)

substitution {x: brown_bat}


True

In [ ]:
for k in pred_map.keys():
    print(type(k))

### Demonstrate solution to Resolution Refutation

In [179]:
def resolution_kb1():
    Bat1 = Predicate("Bat")
    Mammal1 = Predicate("Mammal")
    UsedInBaseball1 = Predicate("UsedInBaseball")
    HasBarrel1 = Predicate("HasBarrel")
    HasWings1 = Predicate("HasWings")
    Flies1 = Predicate("Flies")
    Wing1 = Predicate("Wing")
    Barrel1 = Predicate("Barrel")
    CanFly1 = Predicate("CanFly")
    BrownBat1 = Predicate("BrownBat")

    x = Symbol('x')
    Brown_bat = Symbol('brown_bat')

    Bat = Bat1(x)
    Mammal = Mammal1(x)
    UsedInBaseball = UsedInBaseball1(x)
    HasBarrel = HasBarrel1(x)
    HasWings = HasWings1(x)
    Flies = Flies1(x)
    Wing = Wing1(x)
    Barrel = Barrel1(x)
    CanFly = CanFly1(Brown_bat)
    BrownBat = BrownBat1(Brown_bat)
    Bat2 = Bat1(Brown_bat)
    negated_query = Not(CanFly1(Brown_bat))

    variables = [x]
    skolem_constants = [Brown_bat]

    clause1 = (Not(Bat1(x)), CanFly1(x))
    clause2 = Bat1(Brown_bat)
    literals1 = clause1[0].args[0].args[0].args
    literals2 = clause2.args
    if literals2[0].has(literals2):
        if literals1[1] in variables and literals2[1] in skolem_constants:
            replace_var = literals2[1]
    new_clause2 = (Not(Bat1(Brown_bat)), CanFly1(Brown_bat))
    if new_clause2[0].equals(Not(clause2)):
        resolvent = new_clause2[1]
    if negated_query.equals(Not(resolvent)):
        empty_res = []

    return True, "The brown bat can fly!"

resolution_kb1()

(True, 'The brown bat can fly!')

### Convert to pdf

In [448]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic
!sudo apt-get install texlive
!pip install pandoc
!apt-get install pandoc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono
  fonts-texgyre fonts-urw-base35 libapache-pom-java libcommons-logging-java
  libcommons-parent-java libfontbox-java libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0
  libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13
  lmodern poppler-data preview-latex-style rake ruby ruby-net-telnet
  ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0 rubygems-integration t1utils
  teckit tex-common tex-gyre texlive-base texlive-binaries texlive-latex-base
  texlive-latex-extra texlive-latex-recommended texlive-pictures
  texlive-plain-generic tipa xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf libavalon-framework-java
  libcommons-logging-java-doc lib

In [449]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [451]:
!cp "/content/drive/MyDrive/Colab Notebooks/KB1-cs830-final-course-project-sushma-anand-akoju-updated.ipynb" /content/

In [452]:
%cd /content/
!jupyter nbconvert --to PDF "/content/KB1-cs830-final-course-project-sushma-anand-akoju-updated.ipynb"

/content
[NbConvertApp] Converting notebook /content/KB1-cs830-final-course-project-sushma-anand-akoju-updated.ipynb to PDF
[NbConvertApp] Writing 172458 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 125870 bytes to /content/KB1-cs830-final-course-project-sushma-anand-akoju-updated.pdf
